## Convert to SourceData Format

VERSION: `Process_SourceData_Format`

Compresses a copy of every new session folder, changes permissions, sorts, and moves to the shared projects Folder

Spits out a warning message to deter users that don't know what they are doing. Also logs activity. 

In [ ]:
#!/bin/bash

clear

display_message_box() {
    local msg="$1"
    local border=$(printf '%*s' "${#msg}")
    border=${border// /-}

    tput setaf 1
    echo "+-${border}-+"
    echo "| $msg |"
    echo "+-${border}-+"

    tput sgr0
}

display_message_box "      [WARNING] This action involves sending files to a shared folder. We trust that the data has been properly sorted, cleaned, formatted, and tracked. Ensure that you are choosing the correct project pathways as undoing this action will require manual correction. This execution will not be logged if you abort. Type CONFIRMED to proceed."

read -r confirmation
if [ "$confirmation" != "CONFIRMED" ]; then
    display_message_box "Action Terminated."
    exit 1
fi

echo "Choose from the following projects:"
options=("RestingState_EyeTracking" "SceneViewing_EyeTracking" "Viewpoint_EyeTracking" "Neuropsych_Battery" "Oddity")

select study in "${options[@]}"
do
    case $study in
        "RestingState_EyeTracking"|"SceneViewing_EyeTracking"|"Viewpoint_EyeTracking"|"Neuropsych_Battery"|"Oddity")
            echo "I have selected: $study and I understand its sourcedata structure (Y/N)"
            read -r confirmation
            if [ "$confirmation" != "Y" ]; then
                display_message_box "Action Terminated."
                exit 1
            fi
            break
            ;;
        *) echo "Invalid option $REPLY";;
    esac
done

echo "Project Selected: $study"

projects_base="/home/$USER/projects/def-rolsen/AtRisk-Dataset"
count=0
processed_folders=()

for id_dir in */; do
    if [ -d "$id_dir" ]; then
        id=$(basename "$id_dir")

        for session_dir in "$id_dir"*/; do
            if [ -d "$session_dir" ]; then
                session=$(basename "$session_dir")
                dest_tar_gz_path="$projects_base/$id/$study/$session.tar.gz"

                if [ ! -f "$dest_tar_gz_path" ]; then
                    echo "Processing $session from ID $id in study $study"

                    chmod -R 777 "$session_dir"

                    temp_tar_gz_path="/tmp/$id-$study-$session.tar.gz"

                    tar -czf "$temp_tar_gz_path" -C "$id_dir" "$session"

                    install -D -m 777 "$temp_tar_gz_path" "$dest_tar_gz_path"

                    processed_folders+=("$id/$study/$session")
                    count=$((count+1))
                fi
            fi
        done
    fi
done

echo "Total Processed: $count"

log_to_file() {
    local log_file="/home/$USER/projects/def-rolsen/AtRisk-Dataset/data_log.txt"
    local date_time=$(date "+%Y-%m-%d %H:%M:%S")
    local user=$USER
    local folders=$(IFS=, ; echo "${processed_folders[*]}")

    echo "[$date_time] User: $user, Folders: $folders" >> "$log_file"
}

if [ ${#processed_folders[@]} -ne 0 ]; then
    log_to_file
fi
